# Import Event Log

In [15]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the CSV file
    dataframe_log = pd.read_csv('../../data/extra_logs/large-0.3-3.csv', sep=',')


"""     # Format the dataframe
    dataframe_log = pm4py.format_dataframe(
        dataframe_log,
        case_id='case:concept:name',
        activity_key='concept:name',
        timestamp_key='time:timestamp'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log) """

"     # Format the dataframe\n    dataframe_log = pm4py.format_dataframe(\n        dataframe_log,\n        case_id='case:concept:name',\n        activity_key='concept:name',\n        timestamp_key='time:timestamp'\n    )\n\n    # Convert the dataframe to event log\n    log = log_converter.apply(dataframe_log) "

In [16]:
dataframe_log

,name,timestamp,timestamp_end,anomaly,trace_id,country,day,user
0,Activity A,NaN,NaN,normal,1,Mali,Friday,Ryan
1,Activity AB,NaN,NaN,normal,1,Macao,Friday,Paul
2,Activity AF,NaN,NaN,normal,1,Taiwan,Tuesday,Amanda
3,Activity AC,NaN,NaN,normal,1,Dominican Republic,Monday,Donald
4,Activity AG,NaN,NaN,normal,1,Mauritius,Friday,Keven
...,...,...,...,...,...,...,...,...
54739,Activity AL,NaN,NaN,normal,5000,Austria,Friday,Rossana
54740,Activity AN,NaN,NaN,normal,5000,Cameroon,Friday,Jin
54741,Activity AM,NaN,NaN,normal,5000,Cuba,Monday,Velda
54742,Activity AP,NaN,NaN,normal,5000,Sri Lanka,Thursday,Issac


# Drop unnessary columns

In [17]:
dataframe_log = dataframe_log.drop(columns=['timestamp'])

In [18]:
dataframe_log = dataframe_log.drop(columns=['timestamp_end'])

# Preprocess

In [19]:
codes, uniques = pd.factorize(dataframe_log['name'])
dataframe_log['name'] = codes + 1

In [20]:
codes, uniques = pd.factorize(dataframe_log['day'])
dataframe_log['day'] = codes 

In [21]:
codes, uniques = pd.factorize(dataframe_log['user'])
dataframe_log['user'] = codes

In [22]:
codes, uniques = pd.factorize(dataframe_log['country'])
dataframe_log['country'] = codes

In [23]:
dataframe_log

,name,anomaly,trace_id,country,day,user
0,1,normal,1,0,0,0
1,2,normal,1,1,0,1
2,3,normal,1,2,1,2
3,4,normal,1,3,2,3
4,5,normal,1,4,0,4
...,...,...,...,...,...,...
54739,8,normal,5000,66,0,21
54740,9,normal,5000,41,0,27
54741,10,normal,5000,51,2,17
54742,11,normal,5000,128,4,59


# Generate Prefixes

In [24]:
df_activity = dataframe_log[['name', 'trace_id']]
df_day = dataframe_log[['day', 'trace_id']]
df_user = dataframe_log[['user', 'trace_id']]
df_country = dataframe_log[['country', 'trace_id']]

In [25]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='trace_id', max_len=None, dtype='int32'):
    windows = []
    targets = []
    case_indices = []

    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window.flatten())  # Flatten because we no longer want one-hot encoding
            targets.append(target[0])  # Assume that the target is the first element (activity)
            case_indices.append(case_id)  # Store the case_id corresponding to the window

    if max_len is None:
        max_len = max(len(window) for window in windows)
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype=dtype)

    # Ensure targets and case_indices are numpy arrays
    targets_array = np.array(targets, dtype=dtype)
    case_indices_array = np.array(case_indices, dtype=dtype)

    # Check for length consistency
    assert len(windows_padded) == len(targets_array) == len(case_indices_array), \
        "Length of windows, targets, and case indices arrays must be equal."

    return np.array(windows_padded), targets_array, case_indices_array

In [26]:
windows_activity, targets_activity, case_indices = generate_prefix_windows(df_activity)
windows_day, targets_day, case_indices = generate_prefix_windows(df_day)
windows_user, targets_user, case_indices = generate_prefix_windows(df_user)
windows_country, targets_country, case_indices = generate_prefix_windows(df_country)


# GRU

### Architecture

- Separate Inputs for Each Attribute
- Each attribute is passed through an embedding layer
- Each attribute has its corresponding GRU encoder
- Selective Concatenation: After encoding, the outputs of these GRU layers are concatenated. However, this concatenation is selective, meaning it is structured in a way that prepares the data for effective synthesis without leaking information from the future (next event attributes)
- Decoder GRUs: Integrated Decoding: Post-concatenation, the combined attributes are processed through decoder GRU layers. These layers are tasked with integrating the data from different attributes and preparing it for final prediction. This step is where BINet v3 distinguishes itself by effectively using the interdependencies between different attributes to enhance prediction accuracy.
- Output Layer: Softmax Output for Each Attribute: For each attribute of the next event, a softmax layer predicts a probability distribution over all possible values. This allows the model to output the most likely next event and its attributes based on the learned dependencies and the history encoded by the GRUs.
- E: maximum case length
- We train BINet with a GRU size of 2E (two times the maximum case length)
- on mini batches of size 500 for 20 epochs

In [29]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('trace_id').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, Concatenate, BatchNormalization

def create_binetv3(num_activities, num_days, num_users, num_countries, embedding_dim, gru_units, dropout_rate):
    # Input layers for each attribute
    input_activity = Input(shape=(None,), name='activity_input')
    input_day = Input(shape=(None,), name='day_input')
    input_user = Input(shape=(None,), name='user_input')
    input_country = Input(shape=(None,), name='country_input')

    # Embedding layers for categorical attributes
    embedding_activity = Embedding(input_dim=num_activities, output_dim=embedding_dim, name='activity_embedding')(input_activity)
    embedding_day = Embedding(input_dim=num_days, output_dim=embedding_dim, name='day_embedding')(input_day)
    embedding_user = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(input_user)
    embedding_country = Embedding(input_dim=num_countries, output_dim=embedding_dim, name='country_embedding')(input_country)

    # Encoder GRUs with Batch Normalization for categorical attributes
    encoded_activity = GRU(units=gru_units, return_sequences=True, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)
    encoded_day = GRU(units=gru_units, return_sequences=True, name='day_encoder')(embedding_day)
    bn_day = BatchNormalization(name='bn_day')(encoded_day)
    encoded_user = GRU(units=gru_units, return_sequences=True, name='user_encoder')(embedding_user)
    bn_user = BatchNormalization(name='bn_user')(encoded_user)
    encoded_country = GRU(units=gru_units, return_sequences=True, name='country_encoder')(embedding_country)
    bn_country = BatchNormalization(name='bn_country')(encoded_country)

    # Concatenation of encoded outputs
    concatenated = Concatenate(name='concatenate_encodings')([bn_activity, bn_day, bn_user, bn_country])

    # Decoder GRU
    decoder_output = GRU(units=gru_units, return_sequences=False, name='decoder_gru')(concatenated)
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(decoder_output)

    # Output layers for predicting the next event's attributes
    output_activity = Dense(num_activities, activation='softmax', name='output_activity')(dropout_layer)
    output_day = Dense(num_days, activation='softmax', name='output_day')(dropout_layer)
    output_user = Dense(num_users, activation='softmax', name='output_user')(dropout_layer)
    output_country = Dense(num_countries, activation='softmax', name='output_country')(dropout_layer)

    # Building the model
    model = Model(inputs=[input_activity, input_day, input_user, input_country], outputs=[output_activity, output_day, output_user, output_country])
    model.compile(
        optimizer='adam', 
        loss={
            'output_activity': 'categorical_crossentropy', 
            'output_day': 'categorical_crossentropy',
            'output_user': 'categorical_crossentropy',
            'output_country': 'categorical_crossentropy'
        },
        metrics={
            'output_activity': ['accuracy'], 
            'output_day': ['accuracy'],
            'output_user': ['accuracy'],
            'output_country': ['accuracy']
        }
    )

    return model

# Parameters
gru_units = int(2 * E) 
num_activities = dataframe_log['name'].nunique() + 1
num_days = dataframe_log['day'].nunique() + 1
num_users = dataframe_log['user'].nunique() + 1
num_countries = dataframe_log['country'].nunique() + 1
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3(num_activities, num_days, num_users, num_countries, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 activity_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 day_input (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 user_input (InputLayer)     [(None, None)]               0         []                            
                                                                                                  
 country_input (InputLayer)  [(None, None)]               0         []                      

### Data Splitting

In [36]:
from sklearn.model_selection import train_test_split

# Split the data for the activity attribute
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

# Split the data for the day attribute
train_day, test_day, train_targets_day, test_targets_day = train_test_split(
    windows_day, targets_day, test_size=0.3, random_state=42)

# Split the data for the user attribute
train_user, test_user, train_targets_user, test_targets_user = train_test_split(
    windows_user, targets_user, test_size=0.3, random_state=42)

# Split the data for the country attribute
train_country, test_country, train_targets_country, test_targets_country = train_test_split(
    windows_country, targets_country, test_size=0.3, random_state=42)

### Training

In [37]:
train_targets_activity = train_targets_activity - 1
test_targets_activity = test_targets_activity - 1
train_targets_day = train_targets_day - 1
test_targets_day = test_targets_day - 1
train_targets_user = train_targets_user - 1
test_targets_user = test_targets_user - 1
train_targets_country = train_targets_country - 1
test_targets_country = test_targets_country - 1

In [38]:
from tensorflow.keras.utils import to_categorical

# Convert targets to categorical format
train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities)

train_targets_day_cat = to_categorical(train_targets_day, num_classes=num_days)
test_targets_day_cat = to_categorical(test_targets_day, num_classes=num_days)

train_targets_user_cat = to_categorical(train_targets_user, num_classes=num_users)
test_targets_user_cat = to_categorical(test_targets_user, num_classes=num_users)

train_targets_country_cat = to_categorical(train_targets_country, num_classes=num_countries)
test_targets_country_cat = to_categorical(test_targets_country, num_classes=num_countries)


In [39]:
history = model.fit(
    [train_activity, train_day, train_user, train_country],
    [train_targets_activity_cat, train_targets_day_cat, train_targets_user_cat, train_targets_country_cat],
    validation_data=([test_activity, test_day, test_user, test_country], 
                     [test_targets_activity_cat, test_targets_day_cat, test_targets_user_cat, test_targets_country_cat]),
    epochs=50,
    batch_size=500
)

Epoch 1/50
70/70 [==============================] - 19s 125ms/step - loss: 15.0279 - output_activity_loss: 3.6509 - output_day_loss: 1.8142 - output_user_loss: 4.3920 - output_country_loss: 5.1708 - output_activity_accuracy: 0.1199 - output_day_accuracy: 0.2815 - output_user_accuracy: 0.0593 - output_country_accuracy: 0.0383 - val_loss: 14.3396 - val_output_activity_loss: 3.5450 - val_output_day_loss: 1.7290 - val_output_user_loss: 4.1658 - val_output_country_loss: 4.8998 - val_output_activity_accuracy: 0.0819 - val_output_day_accuracy: 0.2820 - val_output_user_accuracy: 0.0606 - val_output_country_accuracy: 0.0510
Epoch 2/50
70/70 [==============================] - 7s 99ms/step - loss: 11.7332 - output_activity_loss: 2.4912 - output_day_loss: 1.5996 - output_user_loss: 3.5758 - output_country_loss: 4.0665 - output_activity_accuracy: 0.3997 - output_day_accuracy: 0.3453 - output_user_accuracy: 0.1704 - output_country_accuracy: 0.1453 - val_loss: 12.6313 - val_output_activity_loss: 3.02

In [40]:
# Evaluate the model on the validation set
results = model.evaluate(
    [test_activity, test_day, test_user, test_country],
    [test_targets_activity_cat, test_targets_day_cat, test_targets_user_cat, test_targets_country_cat],
    batch_size=64
)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

234/234 [==============================] - 2s 7ms/step - loss: 4.2331 - output_activity_loss: 0.2935 - output_day_loss: 1.0936 - output_user_loss: 1.4559 - output_country_loss: 1.3901 - output_activity_accuracy: 0.9552 - output_day_accuracy: 0.5537 - output_user_accuracy: 0.5267 - output_country_accuracy: 0.5653
Validation Loss: 4.2331156730651855, Validation Accuracy: 0.29353460669517517


In [41]:
# Save the model to an H5 file
model.save('binetv3_Large-3.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

- For each event attribute, BINet's softmax layer outputs a probability distribution over possible values
- The anomaly score for a specific attribute value v is calculated by summing all the probabilities from the softmax output that are greater than the probability assigned to v

In [42]:
# Generate predictions for all inputs
predictions = model.predict([windows_activity, windows_day, windows_user, windows_country])


# Extract predictions for categorical attributes (softmax probabilities)
predictions_activity = predictions[0]
predictions_day = predictions[1]
predictions_user = predictions[2]
predictions_country = predictions[3]

1555/1555 [==============================] - 8s 4ms/step


In [43]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [93]:
# Convert targets to 0-based indices if not already done
targets_activity = targets_activity - 1
targets_day = targets_day - 1
targets_user = targets_user - 1
targets_country = targets_country - 1

In [94]:
# Calculate anomaly scores for each attribute type
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity)
anomaly_scores_day = calculate_anomaly_scores(predictions_day, targets_day)
anomaly_scores_user = calculate_anomaly_scores(predictions_user, targets_user)
anomaly_scores_country = calculate_anomaly_scores(predictions_country, targets_user)

## Insert missing scores for cases with less than 2 Events

In [96]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
score = pd.DataFrame({'case': case_indices})
score['score_day'] = anomaly_scores_day
score['score_activity'] = anomaly_scores_activity
score['score_user'] = anomaly_scores_user
score['score_country'] = anomaly_scores_country


score['case'] = score['case'].astype(int)

score

,case,score_day,score_activity,score_user,score_country
0,1,0.000000,0.0,0.584950,0.807248
1,1,0.000000,0.0,0.000000,0.000000
2,1,0.000000,0.0,0.775203,0.000000
3,1,0.705911,0.0,0.000000,0.875479
4,1,0.563723,0.0,0.527732,0.569008
...,...,...,...,...,...
49739,5000,0.000000,0.0,0.654863,0.995598
49740,5000,0.309263,0.0,0.000000,0.952836
49741,5000,0.000000,0.0,0.554843,0.999843
49742,5000,0.000000,0.0,0.000000,0.966162


In [97]:
import pandas as pd

def contains_all_values(df, column, end):

    # Generate the set of all values in the specified range
    required_values = set(range(1, end + 1))
    
    # Get the unique values in the specified column
    column_values = set(df[column].unique())
    
    # Find missing values
    missing_values = required_values - column_values
    
    # Print missing values if any
    if missing_values:
        print(f"Missing values: {sorted(missing_values)}")
    
    # Check if all required values are in the column values
    return required_values.issubset(column_values)

end = 5000

result = contains_all_values(score, 'case', end)
print(f"Does the 'case' column contain all values between 0 and {end}? {result}")

Does the 'case' column contain all values between 0 and 5000? True


### Threshold (lowest plateau)

In [88]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    thresholds = sorted_scores
    
    # Calculate anomaly ratios for all thresholds at once
    scores_expanded = scores[:, np.newaxis]
    thresholds_expanded = thresholds[np.newaxis, :]
    anomaly_ratios = np.mean(scores_expanded > thresholds_expanded, axis=0)
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        return np.percentile(sorted_scores, 90)

In [98]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    thresholds = sorted_scores
    
    # Calculate anomaly ratios for all thresholds
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        return np.percentile(sorted_scores, 90)

In [99]:
threshold_activity = find_plateaus(anomaly_scores_activity)
threshold_day = find_plateaus(anomaly_scores_day)
threshold_user = find_plateaus(anomaly_scores_user)
threshold_country = find_plateaus(anomaly_scores_country)

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_8310/2145394481.py:21: RuntimeWarning: invalid value encountered in true_divide
  first_derivatives = np.diff(anomaly_ratios) / np.diff(thresholds)


In [158]:
threshold_activity = 0.9

In [175]:
threshold_day = 0.99

In [176]:
threshold_user = 0.99


### Detection

In [177]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [178]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)
labels_day = detect_anomalies(anomaly_scores_day, threshold_day)
labels_user = detect_anomalies(anomaly_scores_user, threshold_user)
labels_country = detect_anomalies(anomaly_scores_country, threshold_country)

# Mapping

In [179]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_activity'] = labels_activity
mapping['predicted_day'] = labels_day
mapping['predicted_user'] = labels_user
mapping['predicted_country'] = labels_country


mapping

,case,predicted_activity,predicted_day,predicted_user,predicted_country
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
49739,5000,0,0,0,0
49740,5000,0,0,0,0
49741,5000,0,0,0,1
49742,5000,0,0,0,0


In [192]:
mapping.head(40)

,case,predicted_activity,predicted_day,predicted_user,predicted_country
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,1,0,0,0,0
9,1,0,0,0,0


In [181]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_activity', 'predicted_day', 'predicted_user', 'predicted_country']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
1       False
2       False
3       False
4       False
5        True
        ...  
4996    False
4997    False
4998    False
4999     True
5000     True
Length: 5000, dtype: bool

# Ground Truth

In [182]:
unique_values = dataframe_log['anomaly'].unique()
print(unique_values)

['normal' 'Rework' 'Attribute' 'Early' 'Late' 'Insert' 'SkipSequence']


- 1: conforming
- 2: non-conforming

In [183]:
# Define the list of strings to check for anomalies
anomaly_strings = ['SkipSequence', 'Insert', 'Early', 'Late', 'Rework']

# Group by 'trace_id' and check if 'anomaly' contains any anomaly strings
def is_anomalous(group):
    return any(label in anomaly_strings for label in group['anomaly'])

# Apply the function to each group and create a new dataframe
anomaly_df = dataframe_log.groupby('trace_id').apply(is_anomalous).reset_index()
anomaly_df.columns = ['trace_id', 'is_anomaly']

# Convert boolean to integer (1 for conforming, 0 for anomaly)
anomaly_df['is_anomaly'] = (~anomaly_df['is_anomaly']).astype(int)

# Extract the conformity array
conformity_array = anomaly_df['is_anomaly']

In [184]:
conformity_array = conformity_array.reset_index(drop=True)
case_prediction = case_prediction.reset_index(drop=True)

In [185]:
# Create a dictionary from the lists
data = {
    'conformity': conformity_array,
    'predicted': case_prediction
}

# Create DataFrame
ground_truth = pd.DataFrame(data)

In [187]:
# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth['predicted'] = 1 - ground_truth['predicted']
ground_truth.head(20)

,conformity,predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,0,0
5,1,1
6,1,0
7,1,0
8,1,0
9,1,0


# Evaluation

In [188]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [189]:
# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.440


In [190]:
# Calculate f1

precision = TP / (TP + FP)
recall = TP / (TP + FN)

f1 = 2 * ((precision * recall) / (precision + recall))
print(f"F1: {f1:.3f}")

F1: 0.424


### Dev (Non Conform Traces)

In [67]:
# Calculate precision for Dev
precision = TN / (TN + FN)
print(f"Precision: {precision:.3f}")

Precision: 0.244


In [68]:
# Calculate recall for Dev
recall = TN / (TN + FP)
print(f"Recall: {recall:.3f}")

Recall: 0.980


### No Dev (Conform Traces)

In [127]:
# Calculate precision for No Dev
precision = TP / (TP + FP)
print(f"Precision: {precision:.3f}")

Precision: 0.982


In [128]:
# Calculate recall for No Dev
recall = TP / (TP + FN)
print(f"Recall: {recall:.3f}")

Recall: 0.470


### AUC-ROC

In [129]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Assuming ground_truth is your DataFrame
# Make sure 'conformity' contains actual labels (0 or 1)
# and 'predicted' contains predicted probabilities or scores
auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
auc_roc

0.7221641471805655

# Trace2Trace Alignments

In [ ]:
# INPUT TRACE 1

bpmn_graph = bpmn_importer.apply("../../data/input_traces/large_trace1.bpmn")

net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

alignments = alignments_petri.apply(log, net, im, fm)

fitness_trace_1 = [trace['fitness'] for trace in alignments]

In [ ]:
# INPUT TRACE 2

bpmn_graph = bpmn_importer.apply("../../data/input_traces/large_trace2.bpmn")

net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

alignments = alignments_petri.apply(log, net, im, fm)

fitness_trace_2 = [trace['fitness'] for trace in alignments]

In [ ]:
# INPUT TRACE 3

bpmn_graph = bpmn_importer.apply("../../data/input_traces/large_trace3.bpmn")

net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

alignments = alignments_petri.apply(log, net, im, fm)

fitness_trace_3 = [trace['fitness'] for trace in alignments]

In [ ]:
# Create a dictionary with the lists
data = {
    'Trace 1': fitness_trace_1,
    'Trace 2': fitness_trace_2,
    'Trace 3': fitness_trace_3
}

# Create the DataFrame
fitness = pd.DataFrame(data)

In [ ]:
# Function to determine the trace with the highest value
def highest_trace(row):
    if row['Trace 1'] == max(row):
        return 'trace_1'
    elif row['Trace 2'] == max(row):
        return 'trace_2'
    else:
        return 'trace_3'

# Add a new column using the highest_trace function
fitness['Closest Trace'] = fitness.apply(highest_trace, axis=1)

In [ ]:
# identify deviation:
# – Skip: One or multiple events are skipped
# – Insert: Random events are inserted
# – Rework: Events are executed multiple times
# – Late: Events are shifted forward
# – Early: Events are shifted backward

import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 1. Import the event log
log = xes_importer.apply("../../data/logs/event_log.xes")

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/large.bpmn")

# 3. Convert the BPMN to a Petri net
net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

# 4. Perform alignment-based conformance checking
alignments = alignments_petri.apply(log, net, im, fm)

# 5. Calculate and print diagnostics
fit_traces = sum(1 for trace in alignments if trace['fitness'] == 1.0)

print(f"Total traces: {len(log)}")
print(f"Conform traces: {fit_traces}")
print(f"Non-Conform traces: {len(log) - fit_traces}")

# 6. Document deviations for each trace
deviations = []

for idx, trace in enumerate(alignments):
    trace_deviations = {
        "trace_index": idx,
        "skip": [],
        "insert": [],
        "rework": [],
        "late": [],
        "early": []
    }
    visited_activities = set()
    alignment_steps = trace['alignment']
    for i, step in enumerate(alignment_steps):
        if step[0] == ">>" and step[1] != ">>":
            trace_deviations["skip"].append(step[1])
        elif step[1] == ">>" and step[0] != ">>":
            trace_deviations["insert"].append(step[0])
        elif step[0] == step[1]:
            if step[0] in visited_activities:
                trace_deviations["rework"].append(step[0])
            visited_activities.add(step[0])
        if step[0] != ">>" and step[1] != ">>" and step[0] != step[1]:
            if alignment_steps[i-1][0] == step[1] or alignment_steps[i-1][1] == step[0]:
                trace_deviations["early"].append(step[0])
            else:
                trace_deviations["late"].append(step[0])
    deviations.append(trace_deviations)

# Print or save the deviations
for dev in deviations:
    print(f"Trace {dev['trace_index']}:")
    print(f"  Skipped Activities: {dev['skip']}")
    print(f"  Inserted Activities: {dev['insert']}")
    print(f"  Rework Activities: {dev['rework']}")
    print(f"  Late Activities: {dev['late']}")
    print(f"  Early Activities: {dev['early']}")
    print("\n")